# JurisCape Node C: The Summarizer (Gemma 2)
**Role**: Summarize raw evidence into 'Fact Sheets' for the Judge.
**Model**: `google/gemma-2-9b-it` (4-bit Quantized).

In [ ]:
# 1. Install Dependencies
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio requests
!pip install torch transformers accelerate bitsandbytes

In [ ]:
# 2. Login to Hugging Face (Required for Gemma)
from huggingface_hub import login
# You need a HF Token with access to Gemma
HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"
try:
    login(token=HF_TOKEN)
except:
    print("Please provide a valid Hugging Face token to load Gemma.")

In [ ]:
# 3. Load Gemma 2 Model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2-9b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
print("Gemma 2 Loaded!")

In [ ]:
# 4. Define Summarization Logic
def generate_summary(text, context_type="evidence"):
    prompt = f"""
    <start_of_turn>user
    Act as a Legal Clerk. Summarize the following {context_type} into a concise list of legally relevant facts.
    Include timestamps if present. Ignore irrelevant conversation.
    
    RAW DATA:
    {text}
    
    OUTPUT (Bullet Points):<end_of_turn>
    <start_of_turn>model
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("model")[-1].strip()


In [ ]:
# 5. Start Server
from pyngrok import ngrok
import uvicorn
import os
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel

NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"
ngrok.set_auth_token(NGROK_TOKEN)

SWARM_SECRET = "change-me-in-prod-secure-swarm-key"
app = FastAPI()

@app.middleware("http")
async def verify_secret(request: Request, call_next):
    if request.url.path in ["/docs", "/openapi.json"]:
        return await call_next(request)
    if request.headers.get("X-Swarm-Secret") != SWARM_SECRET:
        return JSONResponse(status_code=403, content={"detail": "Unauthorized"})
    return await call_next(request)

class SummaryRequest(BaseModel):
    text: str
    context_type: str = "evidence"

@app.post("/summarize")
async def summarize_endpoint(payload: SummaryRequest):
    print(f"Summarizing {len(payload.text)} chars of {payload.context_type}...")
    summary = generate_summary(payload.text, payload.context_type)
    return {"summary": summary}

# CLEANUP & RUN
ngrok.kill()
os.system("pkill ngrok")
tunnel = ngrok.connect(8000)
print(f"\n=== PUBLIC URL: {tunnel.public_url} ===\n")
config = uvicorn.Config(app, port=8000)
await uvicorn.Server(config).serve()